# Restricted versus unrestricted

## High spin states

### Single-reference methods

By a high-spin state, we refer to an open-shell system where all unpaired electrons are chosen to have $\alpha$-spin. As detailed in  section {ref}`sec:spin`, this case is particularly simple since the system can be described by a single Slater determinant.

There are two main ways to proceed:

* Keep the molecular orbitals of the $\alpha$ and $\beta$ spin-orbitals identical but populating the $\alpha$ orbitals with more electrons. This is known as a restricted open-shell calculation.

* Allow for different molecular orbitals for the $\alpha$ and $\beta$ spin-orbitals. This is known as an unrestricted calculation.

Let us illustrate these two approaches on the most important open-shell molecule on earth namely the oxygen molecule.

In [1]:
import multipsi as mtp
import veloxchem as vlx

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 4.


In [2]:
mol_str = """
O   0.0  0.0  -0.6
O   0.0  0.0   0.6
"""
molecule = vlx.Molecule.read_str(mol_str, units="angstrom")

molecule.set_multiplicity(3)  # Set the spin to "triplet"

basis = vlx.MolecularBasis.read(molecule, "cc-pvdz")

rodft_drv = vlx.ScfRestrictedOpenDriver()
rodft_drv.xcfun = "B3LYP"

rodft_results = rodft_drv.compute(molecule, basis)

udft_drv = vlx.ScfUnrestrictedDriver()
udft_drv.xcfun = "B3LYP"

udft_results = udft_drv.compute(molecule, basis)

* Info * Reading basis set from file: /opt/miniconda3/envs/echem/lib/python3.9/site-packages/veloxchem/basis/CC-PVDZ      
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                                  Basis: CC-PVDZ                                                                          
                                                                                                                          
                                  Atom Contracted GTOs          Primitive GTOs                                            
                                                                                                                          
                

Owing to the larger flexibility in the reference state, the UDFT energy is slightly lower the RODFT one.

In [4]:
print(f"ROB3LYP energy: {rodft_drv.get_scf_energy() : 14.8f}")
print(f" UB3LYP energy: {udft_drv.get_scf_energy() : 14.8f}")

ROB3LYP energy:  -150.33015042
 UB3LYP energy:  -150.33393255


In the RODFT case, we have one set of MOs with the first seven being doubly occupied and MOs eight and nine being singly occupied. The orbital energies are:

In [12]:
print(rodft_results["E_alpha"])
print(rodft_results["E_beta"])

[-19.27665036 -19.27649376  -1.28388535  -0.78649766  -0.52368568
  -0.50971915  -0.50971915  -0.20092026  -0.20092026   0.23772183
   0.79594885   0.8347283    0.8347283    0.89859242   0.95034018
   0.95034018   1.07314936   1.69927965   2.08264765   2.08264765
   2.34862061   2.34862145   2.65824564   2.65824657   2.81489628
   3.30062197   3.30062197   3.7801952 ]
[-19.27665036 -19.27649376  -1.28388535  -0.78649766  -0.52368568
  -0.50971915  -0.50971915  -0.20092026  -0.20092026   0.23772183
   0.79594885   0.8347283    0.8347283    0.89859242   0.95034018
   0.95034018   1.07314936   1.69927965   2.08264765   2.08264765
   2.34862061   2.34862145   2.65824564   2.65824657   2.81489628
   3.30062197   3.30062197   3.7801952 ]


In the UHF is case, there are two different sets of MOs with nine occupied $\alpha$-spin orbitals and seven occupied $\beta$-spin orbitals. The orbital energies are:

In [13]:
print(udft_results["E_alpha"])
print(udft_results["E_beta"])

[-19.29060541 -19.29052142  -1.31241964  -0.8307511   -0.56128655
  -0.56128655  -0.54325196  -0.29936529  -0.29936529   0.21614495
   0.78643753   0.80682008   0.80682008   0.89114085   0.92230336
   0.92230336   1.05856796   1.67842059   2.06136016   2.06136016
   2.30837463   2.30837547   2.61252211   2.61252302   2.79245109
   3.27406086   3.27406086   3.76220036]
[-19.26107222 -19.2608452   -1.25525728  -0.74211044  -0.50336112
  -0.45864154  -0.45864154  -0.10410496  -0.10410496   0.26034414
   0.80589793   0.86450688   0.86450688   0.90593529   0.9815685
   0.9815685    1.08853499   1.72074183   2.10456366   2.10456366
   2.38978949   2.38979032   2.70494955   2.70495047   2.83863705
   3.32767442   3.32767442   3.79910186]


The expectation values of the  [total spin operator](https://kthpanor.github.io/echem/docs/elec_struct/spin.html#many-electron-systems) can be determined.

In [16]:
print(f"<S^2> RODFT = {rodft_drv.compute_s2(molecule, rodft_results) : 8.6f}")
print(f"<S^2> UDFT  = {udft_drv.compute_s2(molecule, udft_results) : 8.6f}")

<S^2> RODFT =  2.000000
<S^2> UDFT  =  2.006227


The ground state of oxygen is a triplet with spin quantum number $S = 1$ and

$$
\langle \hat{S}^2 \rangle = S(S+1) = 2
$$ 
 
This is exactly what we get with a restricted open-shell description, but not in the unrestricted case. In general, an unrestricted state is not a eigenfunction of $\hat{S}^2$ and one refers to this situation by saying that the state is spin contaminated. Implications of this will be discussed in more details in the section on low spin open-shell systems where the problem is more pronounced.

The relative simplicity of the unrestricted scheme together with its rather good performance makes it more widely used than the restricted open-shell form.

```{note}
The additional flexibility of unrestricted molecular orbitals does not lower the energy of *closed-shell* restricted states.
```

### Multi-reference methods

A multi-configurational method such as CASSCF can also be used and we note that a CASSCF wave function with only the high-spin open-shell MOs in the active space is equivalent to restricted open-shell Hartree–Fock (ROHF).

In [17]:
rohf_drv = vlx.ScfRestrictedOpenDriver()
rohf_results = rohf_drv.compute(molecule, basis)

# By default, OrbSpace includes all open-shells in the active space
space = mtp.OrbSpace(molecule, rohf_drv.mol_orbs)
mcscf_drv = mtp.McscfDriver()
mcscf_drv.compute(molecule, basis, space)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Open-Shell Hartree-Fock                              
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [20]:
print(f"       ROHF energy: {rohf_drv.get_scf_energy() : 14.8f}")
print(f"CASSCF(2,2) energy: {mcscf_drv.getEnergy() : 14.8f}")

       ROHF energy:  -149.60946112
CASSCF(2,2) energy:  -149.60946112


## Low spin states

### Single-reference methods: broken symmetry

To look at a low spin open-shell, we can continue looking at the oxygen molecule, specifically the singlet states. Looking online, we see that [singlet oxygen](https://en.wikipedia.org/wiki/Singlet_oxygen) has two lowest states, one noted $^1\Delta_g$ at 22.5 kcal/mol above the ground triplet state and one $^1\Sigma^+_g$ at 37.5 kcal/mol above the triplet. Let's first compute the closed-shell singlet with standard (restricted) DFT.

In [7]:
molecule.set_multiplicity(1)  # Switch to singlet

rdft_drv = vlx.ScfRestrictedDriver()
rdft_drv.xcfun = "B3LYP"
r_result = rdft_drv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Kohn-Sham                                            
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [8]:
print("R-DFT relative energy")
print(
    "Compared to RO-DFT triplet:",
    (rdft_drv.get_scf_energy() - rodft_drv.get_scf_energy()) * 627.51,
)
print(
    "compared to U-DFT triplet: ",
    (rdft_drv.get_scf_energy() - udft_drv.get_scf_energy()) * 627.51,
)

R-DFT relative energy
Compared to RO-DFT triplet: 36.92114599675177
compared to U-DFT triplet:  39.29447141163301


While these energies matches reasonably well that of the $^1\Sigma^+_g$, it is not the lowest singlet. There are now 2 distinct ways to reach the lowest singlet using DFT. The first one is to compute the excited states of this singlet using TD-DFT:

In [9]:
rsp_drv = vlx.TDAExciDriver()
rsp_drv.update_settings({"nstates": 1}, {"xcfun": "B3LYP"})
rsp_results = rsp_drv.compute(molecule, basis, r_result)

                                                                                                                          
                                                     TDA Driver Setup                                                     
                                                                                                                          
                               Number of States                : 1                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-04                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-15                                                  
                

In [10]:
print("Excitation energy:", rsp_results["eigenvalues"])
E_S = (
    rdft_drv.get_scf_energy() + rsp_results["eigenvalues"][0]
)  # Energy of the "excited" state
print("Lowest TD-DFT singlet relative energy")
print("Compared to RO-DFT triplet:", (E_S - rodft_drv.get_scf_energy()) * 627.51)
print("compared to U-DFT triplet: ", (E_S - udft_drv.get_scf_energy()) * 627.51)

Excitation energy: [-0.01869682]
Lowest TD-DFT singlet relative energy
Compared to RO-DFT triplet: 25.188702558890068
compared to U-DFT triplet:  27.56202797377131


We can see that the lowest excitation energy is actually negative, implying the existence of a state below the close-shell one we had found using restricted DFT. This is an open-shell singlet.

Let's try to compute this singlet directly. Restricted open-shell will not help, since for a singlet, restricted open-shell is equivalent to restricted. So we will proceed with unrestricted:

In [11]:
singlet_udft_drv = vlx.ScfUnrestrictedDriver()
singlet_udft_drv.xcfun = "B3LYP"
singlet_u_result = singlet_udft_drv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Unrestricted Kohn-Sham                                          
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [12]:
print("Singlet RDFT energy:", rdft_drv.get_scf_energy())
print("Singlet UDFT energy:", singlet_udft_drv.get_scf_energy())

Singlet RDFT energy: -150.27131288130073
Singlet UDFT energy: -150.27131288208315


Disappointingly, there is no difference beyond numerical noise. The problem is that the open-shell unrestricted solution exists, but the closed-shell solution is a stable point. Since there is no magnetic field, $\alpha$ and $\beta$ electrons experience the same force, and there is no reason therefore for the $\alpha$ and $\beta$ orbitals to differ. The minimization is stuck in a symmetric solution. In the high-spin case, this symmetry was broken by the fact that we had more $\alpha$ electrons than $\beta$, thus creating a different field.

To change this and find the lower solution, we need to break the symmetry, and hence, the resulting solution will be called "broken symmetry solution". Codes sometimes offer a keyword to actively break the symmetry. Here we will use the maximum overlap method to create a different starting occupation for the $\alpha$ and $\beta$ orbitals.

In [13]:
# A list of which orbitals are occupied in alpha and beta (starting from the triplet orbitals)
a_occ = [0, 1, 2, 3, 4, 5, 6, 7]
b_occ = [0, 1, 2, 3, 4, 5, 6, 8]
singlet_udft_drv.maximum_overlap(molecule, basis, rdft_drv.mol_orbs, a_occ, b_occ)
singlet_udft_drv.conv_thresh = 1.0e-5
singlet_u_result = singlet_udft_drv.compute(molecule, basis)

                                                                                                                          
* Info * Checkpoint written to file: veloxchem_scf_2022-12-01T22.23.55.scf.h5                                             
                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Unrestricted Kohn-Sham                                          
                   Initial Guess Model             : Restart from Checkpoint                                              
                   Convergence Accelerator         : Direct Inversion of Iterative Subspace                               
                

In [14]:
print("Singlet RDFT energy:", rdft_drv.get_scf_energy())
print("Singlet UDFT energy:", singlet_udft_drv.get_scf_energy())

print(
    "Broken symmetry singlet-triplet gap:",
    (singlet_udft_drv.get_scf_energy() - udft_drv.get_scf_energy()) * 627.51,
)

Singlet RDFT energy: -150.27131288130073
Singlet UDFT energy: -150.31750582825958
Broken symmetry singlet-triplet gap: 10.307935265482534


With this trick, we have now found a lower energy singlet. We can look at the natural orbitals and see that this indeed correspond to two singly occupied orbitals (the two $\pi^*$):

In [15]:
singlet_udft_drv.natural_orbitals().print_orbitals(molecule, basis)

                                                                                                                          
                                                 Spin Restricted Orbitals                                                 
                                                 ------------------------                                                 
                                                                                                                          
               Molecular Orbital No.   4:                                                                                 
               --------------------------                                                                                 
               Occupation: 2.000 Energy:   -1.01238 a.u.                                                                  
               (   1 O   2s  :    -0.32) (   1 O   3s  :    -0.38) (   2 O   2s  :    -0.32)                              
               (

However, the energy gap (10.3 kcal/mol) is well below that from the literature of 22.5 kcal/mol. The reason was hinted in the beginning of this chapter when we mentioned that high spin open-shells are simpler because they are single determinantal. This is not the case for an open-shell singlet, which is a wavefunction made of (at least) two determinants, here for a two-electron case:

$$ | \Psi_S \rangle = \frac{1}{\sqrt{2}} ( | \alpha \beta \rangle -| \beta \alpha \rangle ) $$

in our broken symmetry wavefunction, we have only one determinant. Since the triplet is

$$ | \Psi_T \rangle = \frac{1}{\sqrt{2}} ( | \alpha \beta \rangle +| \beta \alpha \rangle ) $$

we see that a single determinant will be a perfect mix of true singlet and triplet wavefunctions, and this is also reflected in the spin contamination of the UDFT wavefunction:

In [16]:
print(f"<S^2> UDFT  = {singlet_udft_drv.compute_s2(molecule, singlet_u_result):.6}")

<S^2> UDFT  = 1.00328


A true singlet would be 0, and a triplet 2, but here we have a perfect mix, so we get about 1.

However, knowing this fact, we can actually correct the energy. Since we know the energy of the triplet and of the broken-symmetry solution which is a mix of true singlet and triplet, we can estimate the energy of the true singlet. In this case, since the broken symmetry is half singlet half triplet, we just need to multiply the energy by two and remove the triplet energy to get our true singlet energy:

In [17]:
E_T = udft_drv.get_scf_energy()  # triplet energy
E_BS = singlet_udft_drv.get_scf_energy()  # broken-symmetry energy
E_S = 2 * E_BS - E_T
print("Estimated singlet-triplet gap:", (E_S - E_T) * 627.51)

Estimated singlet-triplet gap: 20.61587053096507


This is now very close to the experimental energy of 22.5 kcal/mol.

This formula to obtain the "true" singlet energy is a specific case of the more general "weighted-averaged broken symmetry" (WABS) method, which can in principle correct any broken symmetry solution if you know the corresponding high spin energy and the expectation value $<S^2>$.

### Multi-reference solution

In this case, the multi-reference solution is arguably simpler. You simply need to have an active space containing the right number of orbitals to describe the open-shells. For O$_2$, this is simply 2 electrons in 2 orbitals.

In [18]:
space = mtp.OrbSpace(molecule, rdft_drv.mol_orbs)
space.CAS(2, 2)
singlet_mcscf_drv = mtp.McscfDriver()
singlet_mcscf_drv.compute(molecule, basis, space)


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 7
Number of active orbitals:              2
Number of virtual orbitals:             19

    This is a CASSCF wavefunction: CAS(2,2)

          CI expansion:
          -------------
Number of determinants:      3


                                                                                                                          
        MCSCF Iterations
        ----------------
                                                                                                                          
     Iter. | Average Energy | E. Change | Grad. Norm | CI Iter. |   Time
     ---------------------------------------------------------------------
        1     -149.555806448     0.0e+00      2.6e-01          1   0:00:00
        2     -149.561845052    -6.0e-03      4.4e-02          1   0:00:00
        3     -149.562109449    -2.6e-04      1.3e-02          1   0:00:00
      

In [19]:
print(
    "Lowest singlet relative energy:",
    (singlet_mcscf_drv.getEnergy(0) - mcscf_drv.getEnergy()) * 627.51,
)

Lowest singlet relative energy: 29.699916921469505


The CASSCF automatically finds the open-shell to be the lowest state.

The CASSCF code in multipsi even allows to compute singlets and triplets simultaneously. We only need to deactivate the spin restriction that is applied by defaults to singlets and compute several states at the same time (here 4):

In [20]:
space = mtp.OrbSpace(molecule, rdft_drv.mol_orbs)
space.spinrestricted = False
space.CAS(2, 2)
all_mcscf_drv = mtp.McscfDriver()
all_mcscf_drv.compute(molecule, basis, space, nstates=4)


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 7
Number of active orbitals:              2
Number of virtual orbitals:             19

    This is a CASSCF wavefunction: CAS(2,2)

          CI expansion:
          -------------
Number of determinants:      4


                                                                                                                          
        MCSCF Iterations
        ----------------
                                                                                                                          
     Iter. | Average Energy | E. Change | Grad. Norm | CI Iter. |   Time
     ---------------------------------------------------------------------
        1     -149.555736506     0.0e+00      2.6e-01          1   0:00:00
        2     -149.561864997    -6.1e-03      4.4e-02          1   0:00:00
        3     -149.562122529    -2.6e-04      1.2e-02          1   0:00:00
      

In [21]:
print(
    "First singlet relative energy:",
    (all_mcscf_drv.getEnergy(1) - all_mcscf_drv.getEnergy(0)) * 627.51,
)
print(
    "First singlet relative energy:",
    (all_mcscf_drv.getEnergy(2) - all_mcscf_drv.getEnergy(0)) * 627.51,
)
print(
    "First singlet relative energy:",
    (all_mcscf_drv.getEnergy(3) - all_mcscf_drv.getEnergy(0)) * 627.51,
)

First singlet relative energy: 29.556048789947553
First singlet relative energy: 29.556048791534863
First singlet relative energy: 59.11209758549527


In a single calculation, the CASSCF finds the lowest state to be a triplet (see the spin multiplicity in the output), the following 2 states to be doubly degenerate singlets (the $^1\Delta_g$ state is doubly degenerate) followed by a higher lying singlet. The energies are not a perfect match (CASSCF with such a small active space does not include correlation), but the order is correct.

Note that in this small molecule, a much better agreement with the experiment can be found by simply expanding the active space to include all 2p orbitals of the oxygen (resulting in 6 orbitals):

In [22]:
space.CAS(8, 6)
large_mcscf_drv = mtp.McscfDriver()
large_mcscf_drv.compute(molecule, basis, space, nstates=4)


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 4
Number of active orbitals:              6
Number of virtual orbitals:             18

    This is a CASSCF wavefunction: CAS(8,6)

          CI expansion:
          -------------
Number of determinants:      225


                                                                                                                          
        MCSCF Iterations
        ----------------
                                                                                                                          
     Iter. | Average Energy | E. Change | Grad. Norm | CI Iter. |   Time
     ---------------------------------------------------------------------
        1     -149.640346807     0.0e+00      1.0e-01          4   0:00:00
        2     -149.665981557    -2.6e-02      7.1e-02          4   0:00:00
        3     -149.670480286    -4.5e-03      6.4e-02          4   0:00:00
    

In [23]:
print(
    "First singlet relative energy:",
    (large_mcscf_drv.getEnergy(1) - large_mcscf_drv.getEnergy(0)) * 627.51,
)
print(
    "First singlet relative energy:",
    (large_mcscf_drv.getEnergy(2) - large_mcscf_drv.getEnergy(0)) * 627.51,
)
print(
    "First singlet relative energy:",
    (large_mcscf_drv.getEnergy(3) - large_mcscf_drv.getEnergy(0)) * 627.51,
)

First singlet relative energy: 21.002664914600178
First singlet relative energy: 21.00266491468935
First singlet relative energy: 39.02172559749284


which indeed compares very favourably to the experimental energies:
* $^1\Delta_g$: 22.5 kcal/mol
* $^1\Sigma^+_g$: 37.5 kcal/mol

CASSCF also provides the wavefunctions, allowing us to clearly see the multideterminant nature of the states:

In [24]:
for i, vec in enumerate(all_mcscf_drv.CIVecs):
    print("Wavefunction for state", i + 1)
    print(vec)

Wavefunction for state 1
Determinant     coef.    weight 
ab             -0.707    0.500 
ba              0.707    0.500 

Wavefunction for state 2
Determinant     coef.    weight 
20              0.707    0.500 
02             -0.707    0.500 

Wavefunction for state 3
Determinant     coef.    weight 
ab             -0.707    0.500 
ba             -0.707    0.500 

Wavefunction for state 4
Determinant     coef.    weight 
20             -0.707    0.500 
02             -0.707    0.500 



The determinant notation lists the occupation of the two orbitals in order, with "0" meaning "empty", "a" and "b" meaning respectively singly occupied with an $\alpha$ or $\beta$ electron, and "2" meaning doubly occupied. Clearly in this case, every single state in the calculation is an equal-weight superposition of 2 determinants.